In [20]:
from sklearn import datasets
import numpy as np
from collections import defaultdict
from tqdm import tqdm


iris = datasets.load_iris()
data_ = iris['data']
labels_ = iris['target']
data_ = np.concatenate((data_,labels_.reshape(-1,1)), axis=1)

In [21]:
from math import sqrt, pi, pow, e
def Gaussian_Dist(data, labels):
    cls2mean_var = defaultdict(list)
    cls2counts = defaultdict(int)
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if len(cls2mean_var[labels[i]]) == 0:
                cls2mean_var[labels[i]] = [0.0] * data.shape[1], [0.0] * data.shape[1]
            cls2counts[labels[i]] += 1
            cls2mean_var[labels[i]][0][j] += (data[i, j] - cls2mean_var[labels[i]][0][j])/cls2counts[labels[i]]
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            cls2mean_var[labels[i]][1][j] += pow((data[i, j] - cls2mean_var[labels[i]][0][j]), 2)/cls2counts[labels[i]]


    return cls2mean_var

In [22]:
def pdf(x, m, v):
    return 1/(sqrt(2*pi)*v)*pow(e, -pow(x-m, 2)/(2*v))

In [23]:
cls2mean_var_ = Gaussian_Dist(data_, labels_)

In [24]:
def prediction(cls2mean_var, inputs, pdf_=pdf):
    results = {}
    for cls, mv in cls2mean_var.items():
        results[cls] = 1.0
        for i in range(inputs.shape[0]):
            results[cls] *= pdf_(inputs[i], mv[0][i], mv[1][i])


    return sorted(results.items(), key=lambda x:x[1])[-1][0]

In [25]:
def evaluate(cls2mean_var, test):
    acc = 0.0
    for i in range(test.shape[0]):
        pre = prediction(cls2mean_var, test[i,:-1])
        if pre == test[i, -1]:
            acc += 1
    return acc / test.shape[0]

In [26]:
def cross_validation(data, k):
    size = data.shape[0]//k
    acc = 0.0
    for i in tqdm(range(k)):
        valid = data[i*size: (i+1)*size,...]
        train = np.concatenate((data[: i*size,...],data[(i+1)*size: ,...]), axis=0)
        cls2mean_var = Gaussian_Dist(train, train[...,-1])
        acc += evaluate(cls2mean_var, valid)

    acc /= k
    print("{}-Fold validation result is {}".format(k, acc))

In [43]:
cross_validation(data_, 10)

100%|██████████| 10/10 [00:00<00:00, 974.67it/s]


10-Fold validation result is 0.7733333333333334
